In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torchvision import transforms
from tqdm import *
import matplotlib.pyplot as plt
import copy
from torch.autograd.gradcheck import zero_gradients
import pandas  as pd 
import seaborn as sns
import re
import torch.utils.data as data
import zipfile
from PIL import Image

In [5]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# 解压文件
# ! unzip ./dog_vs_cats/test1.zip 
# ! unzip ./dog_vs_cats/train.zip 

In [7]:
# 图片参数
IMAGE_H = 200
IMAGE_W = 200

In [8]:
data_transform = transforms.Compose([
    transforms.ToTensor()   
])

In [9]:
class DogsVSCatsDataset(data.Dataset):      
    def __init__(self, mode, dir):          
        self.mode = mode
        self.list_img = []                 
        self.list_label = []                
        self.data_size = 0                  
        self.transform = data_transform    

        if self.mode == 'train':            
            dir = './dog_vs_cats/train/'          
            for file in os.listdir(dir):    
                self.list_img.append(dir + file)        
                self.data_size += 1                     
                name = file.split(sep='.')    

                if name[0] == 'cat':
                      self.list_label.append(0)         
                else:
                      self.list_label.append(1)         
        elif self.mode == 'test1':          
            dir = './dog_vs_cats/test1/'            
            for file in os.listdir(dir):
                self.list_img.append(dir + file)    
                self.data_size += 1
                self.list_label.append(2)       
            #  self.list_img.sort(key=lambda x: int(x[20:-4]))
        else:
            return print('Undefined Dataset!')

    def __getitem__(self, item):            
        if self.mode == 'train':                                        
            img = Image.open(self.list_img[item])                       
            img = img.resize((IMAGE_H, IMAGE_W))                       
            img = np.array(img)[:, :, :3]                               
            label = self.list_label[item]                              
            return self.transform(img), torch.LongTensor([label])      
        elif self.mode == 'test1':                                       
            img = Image.open(self.list_img[item])
            # 重置大小
            img = img.resize((200, 200))
            img = np.array(img)[:, :, :3]
            return self.transform(img)                                  
        else:
            print('None')

    def __len__(self):
        return self.data_size             



In [10]:
class Net(nn.Module):                                       
    def __init__(self):                                     
        super(Net, self).__init__()                         
        self.conv1 = torch.nn.Conv2d(3, 16, 3, padding=1)   
        self.conv2 = torch.nn.Conv2d(16, 16, 3, padding=1)  

        self.f1 = nn.Linear(50*50*16, 128)                 
        self.f2 = nn.Linear(128, 64)                       
        self.f3 = nn.Linear(64, 2)                         

    def forward(self, x):                   
        x = self.conv1(x)                   
        x = F.relu(x)                       
        x = F.max_pool2d(x, 2)              

        x = self.conv2(x)                   
        x = F.relu(x)                       
        x = F.max_pool2d(x, 2)              

        x = x.view(x.size()[0], -1)         
        x = F.relu(self.f1(x))            
        x = F.relu(self.f2(x))             
        x = self.f3(x)                     

        return F.softmax(x, dim=1)          


## 训练网络

In [12]:
# train net
from torch.utils.data import DataLoader as DataLoader
import torch
from torch.autograd import Variable
import torch.nn as nn

dataset_dir = './dog_vs_cats/'       # 数据集路径
workers = 2                          # PyTorch读取数据线程数量
batch_size = 16                      # batch_size大小
lr = 0.0001                          # 学习率
Epoch = 4

# 实例化测试数据集
datafile_train = DogsVSCatsDataset('train', dataset_dir)      
dataloader = DataLoader(datafile_train, batch_size=batch_size, shuffle=True, num_workers=workers)    
print('Dataset loaded! length of train set is {0}'.format(len(datafile_train)))

model = Net()                       
# model = model.to(device)               

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=lr)         
criterion = torch.nn.CrossEntropyLoss()                         

cnt = 0             # 训练图片数量

for epoch in range(Epoch):
  cnt = 0 
  for img, label in dataloader:       
    if cnt==100: break
    out = model(img)                                                   
    loss = criterion(out, label.squeeze())      
    loss.backward()                             
    optimizer.step()                            
    optimizer.zero_grad()                       
    cnt += 1
    # print('Frame {0}, train_loss {1}'.format(cnt*batch_size, loss/batch_size))          # 打印一个batch size的训练结果
  print("Epoch {0} finished.".format(epoch))
torch.save(model, 'net.pkl')
print("ok")



Dataset loaded! length of train set is 25000
Epoch 0 finished.
Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.
ok


## 测试网络

In [ ]:
# from getdata import DogsVSCatsDataset as DVCD
# from network import Net
import torch
from torch.autograd import Variable
import numpy as np
import csv
import matplotlib.pyplot as plt
from PIL import Image



dataset_dir = './dog_vs_cats'                 # 数据集路径

f = open("./dog_vs_cats/sampleSubmission.csv","w",encoding="utf-8",newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["id","label"])

model.to(device)                                        
# model.eval()                                        # 设定为评估模式，即计算过程中不要dropout

datafile_test = DogsVSCatsDataset('test1', dataset_dir)               
print('Dataset loaded! length of train set is {0}'.format(len(datafile_test)))
for index in range(0,20):
  img = datafile_test.__getitem__(index)                           # 获取一个图像
  img = img.unsqueeze(0)                                     
  out = model(img)                                           
  if out[0, 0] > out[0, 1]:                   # 猫的概率大于狗
      print('the image is a cat')
      label = 0
  else:                                       # 猫的概率小于狗
      print('the image is a dog')
      label = 1
  print("{0} {1}".format(index+1,label))
  print(datafile_test.list_img[index])
  namefile = datafile_test.list_img[index].split(".")
  print(namefile[])
  csv_writer.writerow([index+1,label])
  img = Image.open(datafile_test.list_img[index])      # 打开测试的图片
  plt.figure('image')                             # 利用matplotlib库显示图片
  plt.imshow(img)
  plt.show()


In [ ]:
r = []
with open("./dog_vs_cats/sampleSubmission.csv",encoding = 'utf-8') as text:
    row = csv.reader(text, delimiter = ',')
    for r in row:
        print(r)

